 # Chapter 11 - A Pythonic Object 

 **Sections with code snippets in this chapter:**
 * [Pythonic Object](#pythonic-object)
 * [Vector Class Redux](#vector-class-redux)
 * [An Alternative Constructor](#an-alternative-constructor)
 * [Classmethod Verus Staticmethod](#classmethod-versus-staticmethodsmethod-versus-staticmethod)
 * [Formatted Display](#formatted-displaymatted-display)
 * [A Hashable Vector2d](#a-hashable-vector2dashable-vector2d)
 * [Supporting positional pattern matching](#supporting-positional-pattern-matching)
 * [Compelete listing of vector2d](#compelete-listing-of-vector2d-version-3)
 * [Private and protected attributes](#private-and-protected-attributes-in-python)
 * [Saving Memory with `__slots__`](#saving-memory-with-__slots__)
 * [Overrinding Class Attributes](#overriding-class-attributes)




**The aim of this chapter is to demonstrate the use of special methods and conventions in the construction of a well-behaved `pythonic class`.**

## Pythonic Object


- "For a library or framework to be Pythonic is to make it as easy and natural as possible
for a Python programmer to pick up how to perform a task." __Martijn Faassen

- This chapter starts when chapter1 (Data Model) ends.

- in Python data model, your user-defined types can behave as naturally as the build-in types.

- We can build user-defined classes that behave as real Python objects.

### Object representation
use some special method that called by related build-in function to getting alternative type representation of object
- repr() # string representation for programmers
- str() # string representation for end users

- bytes() # represent object as byte sequence
- format() # get string display of object

## Vector Class Redux

In order to demonstrate methods used to generate object representation we'll use `Vector2d` class 

In [ ]:
# behaviour we expected
"""
Vector2d instances have several representations
>>> v1 = Vector2d(3, 4) 

>>> print(v1.x, v1.y) 
3.0 4.0

>>>x,y = v1 #__iter__ # unpacking
>>>x,y
(3.0, 4.0)

>>> v1 #__repr__
Vector2d(3.0, 4.0)

>>> v1_clone = eval(repr(v1))  # test repr
>>> v1 == v1_clone #__eq__
True

>>> print(v1) #__str__
(3.0, 4.0)

>>> octets = bytes(v1) #__bytes__
>>> octets 
b'd\\x00\\x00\\x00\\x00\\x00\\x00\\x08@\\x00\\x00\\x00\\x00\\x00\\x00\\x10@' 

>>> abs(v1) #__abs__
5.0

>>> bool(v1), bool(Vector2d(0, 0)) #__bool__
(True, False)
"""

In [1]:
# implement this code based on chapter1 to behave as we expect

from array import array
import math


class Vector2d:
    typecode = 'd'  

    def __init__(self, x, y):
        self.x = float(x)    
        self.y = float(y)

    def __iter__(self):
        return (i for i in (self.x, self.y))  

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)  

    def __str__(self):
        return str(tuple(self))  

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +  
                bytes(array(self.typecode, self)))  

    def __eq__(self, other):
        return tuple(self) == tuple(other) 

    def __abs__(self):
        return math.hypot(self.x, self.y)  

    def __bool__(self):
        return bool(abs(self))  

## An Alternative Constructor

Since we can export a Vector2d as bytes, naturally we need a method that imports a Vector2d from a binary sequence.

In [ ]:
# rebuild a Vector2d from binary representation produced by bytes() 

@classmethod  # modify method, so method can be called directly on class itself rather than instance of class
def frombytes(cls, octests):  # no 'self' argument, instead the class argument passed as first argument
    typecode = chr(octests[0])
    memv = memoryview(octests[1:]).cast(typecode)
    return cls(*memv)  
# use the cls argument by invoking it to build a new instance by takes unpacked memv result as parameter

### classmethod Versus staticmethod

`@classmethod`
- use to define a method that operate on class not on instance 
- changes the way method is called
- receive the class itself as the first argument instead of an instance
- classmethod is very handy and clearly useful


`@staticmethod` 
- changes a method so that it receives no special first argument
- as a simple function that defined in class body, not in module level
- good use cases for staticmethod are very rare in authors experience
- staticmethod is not so useful, module-level functions are simpler

In [3]:
# comparing behvaiors of classmethod vs staticmethod
# both methods return all positinal argument

class Demo:
    @classmethod
    def klassmeth(*args):
        return args
    @staticmethod 
    def statmeth(*args):
        return args 

In [4]:
# no matter how invoke method affected by classmethod decorator
# it receives class as the first argument

Demo.klassmeth()

(__main__.Demo,)

In [5]:
Demo.klassmeth('spam')

(__main__.Demo, 'spam')

In [6]:
# mathod affected by staticmethod act as simple function
Demo.statmeth()

()

In [7]:
Demo.statmeth('spam')

('spam',)

## Formatted Display

Aim of this section is to show the Format Specification Mini-Language are extensible to support user-defined types.

to get string display of objects;  `__format__` is used by:

- `f-strings` 
- `format()` build in function 
- `str.format()` method

 all represent formatting to each type by calling `obj.__format__(format_spec)` method.

the `format_spec` is either:

- second argument in `format(my_obj, format_spec)`
or 
- whatever appears after the colon in a replacement field {} inside an `f-strings` or fmt in fmt.str.format()

In [8]:
brl = 1 / 4.82
brl

0.20746887966804978

In [9]:
# 1 format()
format(brl, '0.4f')

'0.2075'

In [10]:
# 2 fmt.str.format()
'1brl = {rate:0.2f} USD'.format(rate = brl)

'1brl = 0.21 USD'

In [11]:
# 3 f-string
f'1 USD = {brl:0.2f} BRL'

'1 USD = 0.21 BRL'

a format string such {0.mass:5.3e} uses two separate notation:

- the left of the colon -> field_name: arbirtaty -> 0.mass

- after the colon -> formatting specifier -> 5.3e




The notation used in formatting specifier is called **Format Specification Mini-Language**


- A few build-in types have their own representation code in the Format Specification Mini-Language. 
- The Format Specification Mini-Language is **extensible**
    - each class gets to interpret the format_spec arguments as it likes.

for example:

In [12]:
from datetime import datetime 
now = datetime.now()


In [13]:
format(now, '%H:%M:%S')

'22:39:30'

In [14]:
"It's now {:%I:%M %p}".format(now)

"It's now 10:39 PM"

**Note:**
 If a class has no `__format__` the method inherited from object returns `str(my_object).`

Because Vector2d has a ` __str__` this works:

In [15]:
v1 = Vector2d(3, 4)
format(v1)

'(3.0, 4.0)'

In [16]:
#pass a format specifier, object.__format__ raises TypeError:

format(v1, '.3f')

TypeError: unsupported format string passed to Vector2d.__format__

In [ ]:
# This is the result we want
# By implementing our own format mini-language 
""""

>>> v1 = Vector2d(3, 4) 
>>> format(v1)
'(3.0, 4.0)'

>>> format(v1, '.2f') 
'(3.00, 4.00)'

>>> format(v1, '.3e') 
'(3.000e+00, 4.000e+00)'

"""

In [ ]:
# implement __format__
# Use the format built-in to apply the fmt_spec to each vector component

def __format__(self, fmt_spec=''):
    components = (format(c, fmt_spec) for c in self) 
    return '({}, {})'.format(*components)

In [ ]:
# code a simple angle method using the math.atan2() function to get the angle

def angle(self):
    return math.atan2(self.y, self.x)

enhance our `__format__` to produce polar coordinates

In [ ]:
# Vector2d.__format__ take #2 now with polar coordinates

def __format__(self, fmt_spec=''):
    if fmt_spec.endswith('p'):
        fmt_spec = fmt_spec[:-1]
        coords = (abs(self), self.angle())
        outer_fmt = '<{}, {}>'
    else:
        coords = self
        outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)

In [ ]:
"""
With Example 11-6, we get results similar to these:

>>> format(Vector2d(1, 1), 'p') 
'<1.4142135623730951, 0.7853981633974483>' 

>>> format(Vector2d(1, 1), '.3ep') 
'<1.414e+00, 7.854e-01>'

>>> format(Vector2d(1, 1), '0.5fp') 
'<1.41421, 0.78540>'
"""

## A Hashable Vector2d

- aim of this section is to make an object hashable for use in sets and as dict keys
- Our Vector2d instances are unhashable
- to make it hashable must implement `__hash__`
- also need to make vector2d instances *immutable* to ensure they don't modified
- prevent accidental change in attibutes by make them **read-only** properties

In [17]:
# our vector2d instance are unhashbe

v1 = Vector2d(3, 4)
hash(v1)

TypeError: unhashable type: 'Vector2d'

In [18]:
# v1 is unhashable so can't put in a set
set([v1])

TypeError: unhashable type: 'Vector2d'

In [19]:
# anyone can modify attribute , to be immutable should prevent it
v1.x , v1.y

(3.0, 4.0)

In [ ]:
# this is the behvaiour we want 
"""
>>> v1.x=7
Traceback (most recent call last):
...
AttributeError: can't set attribute
"""
# do it by making attributes "read-only"


#### changes need to make Vector2d immutable : private attributes , read-only attributes
- **make attributes "private";** 
    - use two leading underscore(with zero or one trailing underscore)



- **make attributes "read-only"** 
    - use `@property` decorator;
        - allow to define a method that acts like an attribute but is read-only


        - getter method named after public property it exposes
-  in every method that just reads attributes, can use public attribute instead of private attribute 
- @property and read/write properties will covered in chapter 22

In [ ]:
# changes need to make Vector2d immutable

class Vector2d:
    typecode = 'd'
    def __init__(self, x, y):
        self.__x = float(x) 
        self.__y = float(y)

    @property 
    def x(self):
        return self.__x
    
    @property
    def y(self):
        return self.__y
    
    def __iter__(self): # use public attribute in all other methods tthat just read attributes
        return (i for i in (self.x, self.y))


In [ ]:
# implementation of hash
def __hash__(self):
    return hash((self.x, self.y))

- **`__hash__` special method:** 
    - Should return an int.
    - Ideally take into account the hashes of the object attributes that are also used in `__eq__` method.
    - `__hash__` documentation suggests computing the hash of a tuple with the components.

In [21]:
# now have hashable vectors
v1 = Vector2d(3, 4)
v2 = Vector2d(3.1, 4.2)
hash(v1), hash(v2)

(1079245023883434373, 1994163070182233067)

In [22]:
{v1, v2}

{Vector2d(3.0, 4.0), Vector2d(3.1, 4.2)}

## Supporting Positional Pattern Matching

instances are compatible with class keyword pattern

to support positional pattern , add class attribute `__match_args__`
- listing the instance attributes in the order they will be used for positional pattern matching.
- If the class `__init__` has required and optional arguments that are assigned to instance attributes, it may be reasonable to name the required arguments in `__match_args__` but not the optional ones.

In [23]:
# implement keyword pattern for vector2d
def keyword_pattern_demo(v: Vector2d) -> None:
    match v:
        case Vector2d(x=0, y=0):
            print(f'{v!r} is null')
        case Vector2d(x=0):
            print(f'{v!r} is vertical')
        case Vector2d(y=0):
            print(f'{v!r} is horizontal')
        case Vector2d(x=x, y=y) if x==y:
            print(f'{v!r} is diagonal')
        case _:
            print(f'{v!r} is awesome')


In [ ]:
# if you try to use a positional pattern , you get error

"""
>>> case Vector2d(_, 0): 
        print(f'{v!r} is horizontal')

    TypeError: Vector2d() accepts 0 positional sub-patterns (1 given)
"""

In [ ]:
# add class attribute __match_args__ to class
class Vector2d:
    __match_args__ = ('x', 'y')
    
    # etc ....

In [24]:
# implement positinal pattern

def positional_pattern_demo(v: Vector2d) -> None:
    match v:
        case Vector2d(0, 0):
            print(f'{v!r} is null')
        case Vector2d(0):
            print(f'{v!r} is vertical')
        case Vector2d(_, 0):
            print(f'{v!r} is horizontal')
        case Vector2d(x, y) if x==y:
            print(f'{v!r} is diagonal')
        case _:
            print(f'{v!r} is awesome')

## Compelete Listing of Vector2d, Version 3

In [20]:
"""
A 2-dimensional vector class

    >>> v1 = Vector2d(3, 4)
    >>> print(v1.x, v1.y)
    3.0 4.0
    >>> x, y = v1
    >>> x, y
    (3.0, 4.0)
    >>> v1
    Vector2d(3.0, 4.0)
    >>> v1_clone = eval(repr(v1))
    >>> v1 == v1_clone
    True
    >>> print(v1)
    (3.0, 4.0)
    >>> octets = bytes(v1)
    >>> octets
    b'd\\x00\\x00\\x00\\x00\\x00\\x00\\x08@\\x00\\x00\\x00\\x00\\x00\\x00\\x10@'
    >>> abs(v1)
    5.0
    >>> bool(v1), bool(Vector2d(0, 0))
    (True, False)


Test of ``.frombytes()`` class method:

    >>> v1_clone = Vector2d.frombytes(bytes(v1))
    >>> v1_clone
    Vector2d(3.0, 4.0)
    >>> v1 == v1_clone
    True


Tests of ``format()`` with Cartesian coordinates:

    >>> format(v1)
    '(3.0, 4.0)'
    >>> format(v1, '.2f')
    '(3.00, 4.00)'
    >>> format(v1, '.3e')
    '(3.000e+00, 4.000e+00)'


Tests of the ``angle`` method::

    >>> Vector2d(0, 0).angle()
    0.0
    >>> Vector2d(1, 0).angle()
    0.0
    >>> epsilon = 10**-8
    >>> abs(Vector2d(0, 1).angle() - math.pi/2) < epsilon
    True
    >>> abs(Vector2d(1, 1).angle() - math.pi/4) < epsilon
    True


Tests of ``format()`` with polar coordinates:

    >>> format(Vector2d(1, 1), 'p')  # doctest:+ELLIPSIS
    '<1.414213..., 0.785398...>'
    >>> format(Vector2d(1, 1), '.3ep')
    '<1.414e+00, 7.854e-01>'
    >>> format(Vector2d(1, 1), '0.5fp')
    '<1.41421, 0.78540>'


Tests of `x` and `y` read-only properties:

    >>> v1.x, v1.y
    (3.0, 4.0)
    >>> v1.x = 123
    Traceback (most recent call last):
      ...
    AttributeError: can't set attribute 'x'

Tests of hashing:

    >>> v1 = Vector2d(3, 4)
    >>> v2 = Vector2d(3.1, 4.2)
    >>> len({v1, v2})
    2

"""

from array import array
import math

# tag::VECTOR2D_V3_SLOTS[]
class Vector2d:
    __match_args__ = ('x', 'y') 

    typecode = 'd'

    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)

    @property
    def x(self):
        return self.__x

    @property
    def y(self):
        return self.__y

    def __iter__(self):
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __hash__(self):
        return hash((self.x, self.y))

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    def angle(self):
        return math.atan2(self.y, self.x)

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

## Private and "Protected" Attributes in Python

in python there is no way create literaly private variable.

just simple mechansim to prevent accidental overwriting of "private" attribute in a subclass.<br>
assign same name for two attributes one in superclass and one in subclass<br>
 will lead to clobber the attribute used by the method inherited. <br> this would be a pain to debug

to prevent this, named instance attribute with feature called ***name mangling***:
 - Name instance in form two leading underscore and zero or at most one trailing underscore 
 - Name mangling, changes the form instances saved in `__dict__`

In [2]:
# this cell should run before run "comlpelete listing of vector2d version3" to work correctly
# attribute names in __dict__ before mangeled 
v1 = Vector2d(3, 4)
v1.__dict__

{'x': 3.0, 'y': 4.0}

In [25]:
# private attribute names are "mangled" by prefixing the _ and the class name
v1 = Vector2d(3, 4)
v1.__dict__

{'_Vector2d__x': 3.0, '_Vector2d__y': 4.0}

In [26]:
# anyone can access private attribute so that is not really private 
# just make them safe from accidental access

v1._Vector2d__x

3.0

- name mangling is about **safety not security**


- anyone who knows how private names are mangled can read the private attribute directly and assign a value to private attribute that mangled.
 >>> `v1._Vector2d__x = 7`


![figure 1-2](https://zehtab.ir/fluentpython%20fig11.png)

A cover on a switch is a safety device, not a security one: it prevents accidents, not sabotage.

- not all python programers use name mangling 
- some prefer use just protected attribute with only one underscore prefix
- single underscore prefix does not have any meaning to the python intrepreter when used in attribute names (don't change the form instance saved in `__dict__`)
- just a convention that you should not access such attributes from outside the class

## Saving memory with `__slots__`

`__slots__` save memory by:
- prevent dynamic creation of instance attributes in dictionary that is memory intenstive
- restrict the instance attributes to only those explicity listed in `__slots__`

##### why using `__slots__`
- By default python stores attributes of each instance in a dict named `__dict__`
- Dict has significant memory overhead
- class attribute `__slots__` holding a sequence of attribute (hidden array of refrence) that use less memory than a `dict`
##### how implement `__slots__`
- `__slots__` must be present when the class is created. adding or changing it later has no effect.
- write attribute name in `tuple` or `list` in `__slots__` (preferably do it with tuple)

##### effect of `__slots__`
- prevent having `__dict__` for instance of a class
- instance will only be able to have attributes listed in `__slots__`
- you must redeclare `__slots__` in each subclass to prevent their instance from having `__dict__`


##### caution of using `__slots__`
- when `__slots__` defined, weak reference support disabled. 
    - unless add `__weakref__` in __slots__.
- if you want use `@cached_property` decorator:
    - it is neccessary to add `__dict__` in `__slots__` (it can entirely defeat optimization and memory saving)



**NOTE:** <br>

Because using `__slots__` has side effects, it really makes sense only when handling a very large number of instances—think millions of instances, not just thousands. In many such cases, using `pandas` may be the best option.

In [27]:
# Pixel class uses __slots__

class Pixel:
    __slots__ = ('x', 'y')

p = Pixel()
p.__dict__ # instances of Pixel have no __dict__

AttributeError: 'Pixel' object has no attribute '__dict__'

In [28]:
p.x = 10
p.y = 20
p.color = 'red' # color attribute not listed in __slots__

AttributeError: 'Pixel' object has no attribute 'color'

In [29]:
# study subclass of Pixel 

class OpenPixel(Pixel):
    pass
op =  OpenPixel()
op.__dict__

{}

In [30]:
op.x = 8
op.__dict__

{}

In [31]:
op.x

8

In [32]:
op.color = 'green' # if set attribute not in __slots__ it stores in instance__dict__
op.__dict__

{'color': 'green'}

To make sure instance of subclass have no `__dict__`: 
- must declare `__slots__` again in the subclass 

In [33]:
class ColorPixel(Pixel):
    __slots__ = ('color',)
cp = ColorPixel()


In [34]:
cp.__dict__

AttributeError: 'ColorPixel' object has no attribute '__dict__'

In [35]:
# when implement __slots__ in both super and subclass we can set attributes declared in the __slots__ of subclass and superclass but no other.
cp.x = 2 
cp.color = 'blue' 
cp.flavor = 'banana'


AttributeError: 'ColorPixel' object has no attribute 'flavor'

### Simple Measure of `__slot__` Savings

In [ ]:
# Example 11-16
class vector2d:
    __match_args__ = ('x', 'y')
    __slots__ = ('__x', '__y')

    typecode = 'd'
    #method are the same as the previous version

In [ ]:
#Example 11-17. mem_test.py creates 10 million Vector2d instances using the class defined in the named module
"""
$ time python3 mem_test.py vector2d_v3
Selected Vector2d type: vector2d_v3.Vector2d
Creating 10,000,000 Vector2d instances
Initial RAM usage:      6,983,680
  Final RAM usage:  1,666,535,424
real 0m11.990s
user 0m10.861s
sys 0m0.978s


$ time python3 mem_test.py vector2d_v3_slots
Selected Vector2d type: vector2d_v3_slots.Vector2d
Creating 10,000,000 Vector2d instances
Initial RAM usage:      6,995,968
  Final RAM usage:    577,839,104
real 0m8.381s
user 0m8.006s
sys 0m0.352s
"""

## Overriding Class Attributes

A distinctive feature of Python is that how class attributes can be used as default values for instance attributes.

for example `typecode` class attribute in vector2d
- if instance created without a `typecode` attribute of their own, `self.typecode` will get `typecode` class attribute by default.
- if you write an instance attribute that are not exist you create new instance attribute(`typecode` instance attribute) then `self.typecode` retrived the instance typecode

**this opens posibility of customizing an individual instance with a different typecode.**


In [36]:
# customizing an instance by setting typecode attribute
# f typecode means each component represent as 4-byte single precision float 
# d typecode means each component represent as 8-byte double precision float (vector2d class typecode = 'd')
v1 = Vector2d(1.1, 2.2)
dumpd = bytes(v1)
dumpd

b'd\x9a\x99\x99\x99\x99\x99\xf1?\x9a\x99\x99\x99\x99\x99\x01@'

In [37]:
len(dumpd)

17

In [38]:
# set the typecode in the v1 instance 
v1.typecode = 'f'
dumpf = bytes(v1)
dumpf

b'f\xcd\xcc\x8c?\xcd\xcc\x0c@'

In [39]:
len(dumpf)

9

In [40]:
# vector2d.typecode is unchanged; only the v1 instance uses typecode 'f'

Vector2d.typecode

'd'

In [ ]:
# if want change class attribute, must set it on the class directly

"""""
 Vector2d.typecode = 'f'
 
"""""

#### Pythonic way for customizing: 
##### use subclass to customize a class attribue : 
class attributes are public , they are inherited by subclass.

- more premanent effect
- more explicit about the change

create subclass just to overwrite the default class attribute 

In [41]:
# create ShortvVector2d as a Vector2d subclass just to overwrite the typecode class attribute

class Shortvector2d(Vector2d):
    typecode = 'f'

sv = Shortvector2d(1/11, 1/27)
sv

Shortvector2d(0.09090909090909091, 0.037037037037037035)

In [42]:
len(bytes(sv))

9

This example justify why we wrote vector2d class_name in `__repr__` in form `type(self).__name__` but not in hardcode form <br>
if hardcoded the class_name , then had to overwrite `__repr__` to just change the classname.<br>
but now, by reading the name form the type of the instance, we made `__repr__` safer to inherit

# Lecturers 

1. Ilya Izadifar [Linkedin](https://www.linkedin.com/in/izadifar)
2. Aidin Zehtab [Linkedin](https://linkedin.com/in/aidin-zehtab)


present date : 2023-01-12

# Reviewers
1. Mahya Asgarian, review date: 2023-11-30 [LinkedIn](https://www.linkedin.com/in/mahya-asgarian-9a7b13249)
2. Saeed Hemmati, review date: 2023-12-1 [LinkedIn](https://www.linkedin.com/in/saeed-hemati/)
3.  Mohammad Ansarifard, review date : 2023-12-1 [Linkedin](https://www.linkedin.com/in/mohammad-ansarifard)